# PyCity Schools Analysis

* As a whole, district schools see better grades and passing rates in reading than in math (Average Score for math vs reading: 76.95 vs 80.96, Average Passing Rate for math vs reading: 67% vs 81%), while charter students perform similarly in both subjects.

* As a whole, charter schools tend to have fewer students, and better performance, than district schools. Based on overall passing rate, the top 5 schools are charter schools, while the bottom 5 schools are district schools.

## District Summary

In [1]:
import pandas as pd
import numpy as np

school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
# Calculate the total number of schools
total_schools_district= school_data["school_name"].count()

In [3]:
# Calculate the total number of students
total_students_district= school_data_complete["student_name"].count()

In [4]:
# Calculate the total budget
total_budget_district = school_data["budget"].sum()

In [5]:
# Calculate the average math score
math_average_district = student_data["math_score"].mean()

In [6]:
# Calculate the average reading score
reading_average_district = student_data["reading_score"].mean()

In [7]:
# Calculate the percentage of students with a passing math score (70 or greater)
passing_math_district = student_data.loc[student_data["math_score"] >= 70, ["Student ID", "student_name", "gender", "grade", "school_name", "reading_score", "math_score"]]
passing_math_count_district = passing_math_district["student_name"].count()
passing_math_percent_district = passing_math_count_district/total_students_district

In [8]:
# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_district= student_data.loc[student_data["reading_score"] >= 70, ["Student ID", "student_name", "gender", "grade", "school_name", "reading_score", "math_score"]]
passing_reading_count_district = passing_reading_district["student_name"].count()
passing_reading_percent_district = passing_reading_count_district/total_students_district

In [9]:
# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
passing_rate_district = (passing_math_percent_district + passing_reading_percent_district)/2

In [10]:
# Create a dataframe to hold the above results
data_district = [['Total Number of Schools',total_schools_district],['Total Students',total_students_district],['Total Budget',total_budget_district],
        ['Average Math Score',math_average_district],['Average Reading Score', reading_average_district], 
        ['% Passing Math', passing_math_percent_district],
        ['% Passing Reading', passing_reading_percent_district],
        ['% Passing Overall', passing_rate_district],]
district_summary = pd.DataFrame(data_district)

In [11]:
# Cleaner Output

data = [['Total Number of Schools',total_schools_district],['Total Students','{:,}'.format(total_students_district)],['Total Budget','${:,.0f}'.format(total_budget_district)],
        ['Average Math Score',round(math_average_district,(2)) ],['Average Reading Score', round(reading_average_district,(2)) ], 
        ['% Passing Math', '{:.2%}'.format(passing_math_percent_district)],
        ['% Passing Reading', '{:.2%}'.format(passing_reading_percent_district)],
        ['% Passing Overall', '{:.2%}'.format(passing_rate_district)]]
new_index= ['Total Number of Schools','Total Students','Total Budget','Average Math Score','Average Reading Score', 
            'Overall Average Score', "% Pasing Math",
            '% Passing Reading']
clean_summary = pd.DataFrame(data)
clean_summary.columns = ['Metrics', 'Values']
clean_summary

,Metrics,Values
0,Total Number of Schools,15
1,Total Students,"39,170"
2,Total Budget,"$24,649,428"
3,Average Math Score,78.99
4,Average Reading Score,81.88
5,% Passing Math,74.98%
6,% Passing Reading,85.81%
7,% Passing Overall,80.39%


## School Summary

In [12]:
# Per Student Budget
school_data["per_student"] = school_data["budget"]/school_data["size"]

In [13]:
# Average Math Score
school_data_complete
grouped_data = school_data_complete.groupby(['school_name']).sum()
math_sum = pd.DataFrame(grouped_data["math_score"])
math_average = pd.merge(math_sum, school_data, on=["school_name"])
math_score = math_average['math_score']/math_average['size']
math_average["average_math_score"]= math_score

In [14]:
# Average Reading Score
reading_sum = pd.DataFrame(grouped_data["reading_score"])
reading_average = pd.merge(reading_sum, school_data, on=["school_name"])
reading_score = reading_average['reading_score']/reading_average['size']
reading_average["average_reading_score"]= reading_score

In [15]:
# Add Average Reading and Math Score to school_data
school_data_math = pd.merge(school_data, math_average, how="left", on=["school_name", "school_name"])
school_data_1 = school_data_math[['school_name', 'type_x','size_x','budget_x','per_student_x','average_math_score']]
school_data_2 = pd.merge(school_data_1, reading_average, how="left", on=["school_name", "school_name"])
school_data_3 = school_data_2[['school_name', 'type_x','size_x','budget_x','per_student_x','average_math_score','average_reading_score']]

In [16]:
# Calculate % Passing Math
math_passers = student_data.loc[student_data['math_score'] >= 70]
math_passers_grouped = math_passers.groupby('school_name').count()
math_passers_grouped
sorted_df = school_data.sort_values('school_name')
sorted_df
new_df = sorted_df[['school_name','size']]
new_df['passed_math_total'] = list(math_passers_grouped['math_score'])
new_df['% Passing Math']= new_df['passed_math_total']/new_df['size']
school_data['% Passing Math'] = new_df['% Passing Math']

/Users/pamstadler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pamstadler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
# Calcuate % Passing Reading
reading_passers = student_data.loc[student_data['reading_score'] >= 70]
reading_passers_grouped = reading_passers.groupby('school_name').count()
reading_passers_grouped
new_df['passed_reading_total'] = list(reading_passers_grouped['reading_score'])
new_df['% Passing Reading']= new_df['passed_reading_total']/new_df['size']
school_data['% Passing Reading'] = new_df['% Passing Reading']

/Users/pamstadler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [18]:
#Overall Passing Rate (Average of the above two) i.e. (avg. math score + avg. reading score)/2
overall_passing_school = (school_data['% Passing Math'] + school_data['% Passing Reading'])/2
school_data['% Overall Passing Rate'] = overall_passing_school

In [19]:
school_summary = school_data_3[["school_name", "type_x", "size_x", "budget_x","per_student_x","average_math_score","average_reading_score"]]
school_summary_names = school_summary.rename(columns={'school_name' : 'School Name', 
                                                      'type_x': 'School Type', 
                                                      'size_x':'Total Students',
                                                      'budget_x':'Total Student Budget',
                                                      'per_student_x' : "Per Student Budget",
                                                      'average_math_score' : "Average Math Score",
                                                      'average_reading_score' : "Average Reading Score"})
school_summary_names["% Passing Math"] = (school_data['% Passing Math'] * 100)
school_summary_names["% Passing Reading"] = (school_data['% Passing Reading'] * 100)
school_summary_names['% Overall Passing Rate'] = (school_data['% Overall Passing Rate'] * 100)
school_summary_names

,School Name,School Type,Total Students,Total Student Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

In [20]:
top_school_summary_names = school_summary_names.sort_values(["% Overall Passing Rate"], ascending=False)
top_school_summary_names.head(5).style.format({'% Passing Math' : "{:.2%}", '% Passing Reading' : "{:.2%}",
                                       '% Overall Passing Rate' : "{:.2%}",'Total Student Budget' : '${:,.2f}',
                                       'Per Student Budget' : '${:,.2f}'})

,School Name,School Type,Total Students,Total Student Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.0619,83.9758,9413.35%,9703.98%,9558.67%
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4183,83.8489,9327.22%,9730.89%,9529.05%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,9459.46%,9594.59%,9527.03%
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.3515,83.8168,9339.24%,9713.90%,9526.57%
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.2742,83.9895,9386.77%,9653.96%,9520.37%


## Bottom Performing Schools (By Passing Rate)

In [21]:
bottom_school_summary_names = school_summary_names.sort_values(["% Overall Passing Rate"])
bottom_school_summary_names.head(5).style.format({'% Passing Math' : "{:.2%}", '% Passing Reading' : "{:.2%}",
                                       '% Overall Passing Rate' : "{:.2%}",'Total Student Budget' : '${:,.2f}',
                                       'Per Student Budget' : '${:,.2f}'})

,School Name,School Type,Total Students,Total Student Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8427,80.7447,6636.66%,8022.01%,7329.33%
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7118,81.158,6598.85%,8073.92%,7336.39%
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.6294,81.1827,6568.39%,8131.64%,7350.02%
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.0725,80.9664,6605.76%,8122.24%,7364.00%
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.1026,80.7463,6830.96%,7929.90%,7380.43%


## Math Scores by Grade

In [65]:
grouped_data_grade = school_data_complete.groupby(['school_name','grade']).sum()
math_sum_grade = pd.DataFrame(grouped_data_grade["math_score"])
grouped_data_grade_count = school_data_complete.groupby(['school_name','grade']).count()
math_average_grade = pd.merge(math_sum_grade, grouped_data_grade_count, on=['school_name', 'grade'])
math_score_grade = math_average_grade['math_score_x']/math_average_grade['size']
math_average_grade["average_math_score_grade"]= math_score_grade
school_summary_grade = math_average_grade[["average_math_score_grade"]]
school_summary_grade.unstack(level=-1)

average_math_score_grade                        \
grade                                     10th       11th       12th   
school_name                                                            
Bailey High School                   76.996772  77.515588  76.492218   
Cabrera High School                  83.154506  82.765560  83.277487   
Figueroa High School                 76.539974  76.884344  77.151369   
Ford High School                     77.672316  76.918058  76.179963   
Griffin High School                  84.229064  83.842105  83.356164   
Hernandez High School                77.337408  77.136029  77.186567   
Holden High School                   83.429825  85.000000  82.855422   
Huang High School                    75.908735  76.446602  77.225641   
Johnson High School                  76.691117  77.491653  76.863248   
Pena High School                     83.372000  84.328125  84.121547   
Rodriguez High School                76.612500  76.395626  77.690748   
Shelton High School                  82.917411  83.383495  83.778976   
Thomas High School                   83.087886  83.498795  83.497041   
Wilson High School                   83.724422  83.195326  83.035794   
Wright High School                   84.010288  83.836782  83.644986   

                                  
grade                        9th  
school_name                       
Bailey High School     77.083676  
Cabrera High School    83.094697  
Figueroa High School   76.403037  
Ford High School       77.361345  
Griffin High School    82.044010  
Hernandez High School  77.438495  
Holden High School     83.787402  
Huang High School      77.027251  
Johnson High School    77.187857  
Pena High School       83.625455  
Rodriguez High School  76.859966  
Shelton High School    83.420755  
Thomas High School     83.590022  
Wilson High School     83.085578  
Wright High School     83.264706

## Reading Score by Grade 

In [66]:
reading_sum_grade = pd.DataFrame(grouped_data_grade["reading_score"])
reading_average_grade = pd.merge(reading_sum_grade, grouped_data_grade_count, on=['school_name', 'grade'])
reading_score_grade = reading_average_grade['reading_score_x']/reading_average_grade['size']
reading_average_grade["average_reading_score_grade"]= reading_score_grade
school_summary_grade_reading = reading_average_grade[["average_reading_score_grade"]]
school_summary_unstacked=school_summary_grade_reading.unstack(level=-1)
school_summary_unstacked

average_reading_score_grade                        \
grade                                        10th       11th       12th   
school_name                                                               
Bailey High School                      80.907183  80.945643  80.912451   
Cabrera High School                     84.253219  83.788382  84.287958   
Figueroa High School                    81.408912  80.640339  81.384863   
Ford High School                        81.262712  80.403642  80.662338   
Griffin High School                     83.706897  84.288089  84.013699   
Hernandez High School                   80.660147  81.396140  80.857143   
Holden High School                      83.324561  83.815534  84.698795   
Huang High School                       81.512386  81.417476  80.305983   
Johnson High School                     80.773431  80.616027  81.227564   
Pena High School                        83.612000  84.335938  84.591160   
Rodriguez High School                   80.629808  80.864811  80.376426   
Shelton High School                     83.441964  84.373786  82.781671   
Thomas High School                      84.254157  83.585542  83.831361   
Wilson High School                      84.021452  83.764608  84.317673   
Wright High School                      83.812757  84.156322  84.073171   

                                  
grade                        9th  
school_name                       
Bailey High School     81.303155  
Cabrera High School    83.676136  
Figueroa High School   81.198598  
Ford High School       80.632653  
Griffin High School    83.369193  
Hernandez High School  80.866860  
Holden High School     83.677165  
Huang High School      81.290284  
Johnson High School    81.260714  
Pena High School       83.807273  
Rodriguez High School  80.993127  
Shelton High School    84.122642  
Thomas High School     83.728850  
Wilson High School     83.939778  
Wright High School     83.833333

## Scores by School Spending

In [24]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [25]:
school_summary_names["Spending Ranges (Per Student)"] = pd.cut(school_summary_names["Per Student Budget"], spending_bins, labels=group_names)
school_summary_names

school_group_spend = school_summary_names.groupby("Spending Ranges (Per Student)")
school_group_spend[["Average Math Score", "Average Reading Score", '% Passing Math',"% Passing Reading", "% Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

In [26]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [27]:
school_summary_names["School Size"] = pd.cut(school_summary_names["Total Students"], size_bins, labels=group_names)
school_group_size = school_summary_names.groupby("School Size")
school_group_size[["Average Math Score", "Average Reading Score", '% Passing Math',"% Passing Reading", "% Overall Passing Rate"]].mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [28]:
school_group_type = school_summary_names.groupby("School Type")
school_group_type[["Average Math Score", "Average Reading Score", '% Passing Math',"% Passing Reading", "% Overall Passing Rate"]].mean()



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
